In [62]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [63]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [64]:
tf.keras.backend.clear_session()

In [65]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [66]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 13 #Referente aos 14 anos de input  

In [67]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Santa Catarina - IDH Renda,Santa Catarina - IDH Longevidade,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,...,Taxa Selic (%),Santa Catarina - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - Consumo de Cimento (t)
0,2008,0.790219,0.851202,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,354.241699,0.797988,...,11.760983,0.755590,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,2653.587000
1,2009,0.787470,0.854807,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,431.347926,0.799099,...,9.472585,0.758432,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,2669.400000
2,2010,0.784890,0.858161,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,792.441715,0.800218,...,9.332485,0.761341,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,2852.759000
3,2011,0.782462,0.861283,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,1268.083588,0.801347,...,10.985689,0.764319,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,3087.810000
4,2012,0.780170,0.864193,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,1549.260439,0.802494,...,8.159014,0.767371,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,3268.310000
5,2012,0.775917,0.869299,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,1549.260439,0.803935,...,8.159014,0.771187,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,3268.310000
6,2013,0.776910,0.868574,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,1645.417258,0.804929,...,7.913561,0.773786,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,3466.227000
7,2014,0.777652,0.867205,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,1598.981415,0.805257,...,10.350442,0.775185,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,3246.749286
8,2015,0.769407,0.877280,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,1488.521029,0.807829,...,12.454398,0.781469,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,3017.084229
9,2016,0.769444,0.875578,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,1430.320858,0.807723,...,13.111908,0.782603,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,2776.217486


In [68]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Santa Catarina - IDH Renda,Santa Catarina - IDH Longevidade,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Santa Catarina - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.790219,0.851202,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,354.241699,0.797988,5.483335e+08,...,2.868108,11.760983,0.755590,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136
1,0.787470,0.854807,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,431.347926,0.799099,5.302764e+08,...,2.305885,9.472585,0.758432,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407
2,0.784890,0.858161,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,792.441715,0.800218,5.046425e+08,...,1.958246,9.332485,0.761341,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779
3,0.782462,0.861283,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,1268.083588,0.801347,4.854622e+08,...,2.079088,10.985689,0.764319,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706
4,0.780170,0.864193,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,1549.260439,0.802494,4.867661e+08,...,2.183282,8.159014,0.767371,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605
5,0.775917,0.869299,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,1549.260439,0.803935,4.867661e+08,...,2.183282,8.159014,0.771187,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605
6,0.776910,0.868574,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,1645.417258,0.804929,5.225848e+08,...,2.403445,7.913561,0.773786,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329
7,0.777652,0.867205,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,1598.981415,0.805257,5.676413e+08,...,2.748397,10.350442,0.775185,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238
8,0.769407,0.877280,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,1488.521029,0.807829,5.179392e+08,...,2.501529,12.454398,0.781469,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937
9,0.769444,0.875578,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,1430.320858,0.807723,4.524144e+08,...,2.073464,13.111908,0.782603,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965


In [69]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3268.310000
5     3466.227000
6     3246.749286
7     3017.084229
8     2776.217486
9     2891.489000
10    2976.485000
11    3258.027000
12    3537.032000
13    4089.333000
14    4255.808000
15            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [70]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - IDH Renda,Santa Catarina - IDH Longevidade,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Santa Catarina - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.827942,-1.956001,-1.847101,-0.139205,-0.846292,-2.034737,1.456899,-2.139725,-1.833164,1.261808,...,2.126822,1.059506,-1.729816,1.521918,-0.711336,-0.537062,-0.951463,-0.367954,0.575264,1.413168
1,1.440880,-1.477357,-1.698747,0.453688,-0.814842,-1.933202,1.479718,-1.960862,-1.482217,0.862198,...,0.115581,-0.002029,-1.435042,0.963040,-2.003505,-1.514401,-1.019885,0.224462,1.119798,1.314572
2,1.077559,-1.032101,-1.128699,1.490705,0.643986,-1.025234,1.451259,-1.123233,-1.129185,0.294908,...,-1.128027,-0.067018,-1.133394,0.652092,-1.445955,-0.702033,-1.644481,-0.850438,0.825912,0.960226
3,0.735681,-0.617605,-0.615309,1.314952,1.424756,-0.376764,1.440971,-0.019888,-0.772504,-0.129560,...,-0.695740,0.699865,-0.824491,-0.137517,-0.782672,0.478560,-1.327704,-1.141323,0.297945,0.197862
4,0.412948,-0.231244,-0.154925,1.035302,1.368304,0.116897,-1.106876,0.632357,-0.410608,-0.100704,...,-0.323007,-0.611365,-0.507955,-0.695220,-0.282901,1.237535,-0.529592,-0.241635,0.302498,-0.929094
5,-0.185858,0.446664,-0.154925,1.035302,1.368304,0.116897,-1.106876,0.632357,0.044335,-0.100704,...,-0.323007,-0.611365,-0.112098,-0.695220,-0.282901,1.237535,-0.529592,-0.241635,0.302498,-0.929094
6,-0.046036,0.350395,0.250875,0.362442,0.933412,0.505675,-0.770707,0.855412,0.358321,0.691978,...,0.464584,-0.725225,0.157460,-0.038362,0.217504,1.364807,0.487249,1.272231,0.635907,-1.764369
7,0.058504,0.168655,0.575454,-0.076378,0.417611,0.699946,-0.515949,0.747695,0.461693,1.689098,...,1.698578,0.405188,0.302547,0.618847,0.160976,-1.005199,0.952293,2.101893,0.910184,-1.457376
8,-1.102396,1.506270,0.781895,-0.497390,-0.994014,0.620023,-0.294912,0.491460,1.273851,0.589170,...,0.815461,1.381165,0.954281,0.436684,0.581481,-1.626436,0.530977,1.028294,0.404370,-0.587689
9,-1.097184,1.280319,0.950337,-1.111184,-1.334058,0.698845,-0.290250,0.356453,1.240247,-0.860922,...,-0.715858,1.686169,1.071849,0.691891,1.107822,-0.046942,0.574009,0.042456,-0.491530,0.084169


In [71]:
reshaped_train_input = dfToInputRNN(train_input)

In [72]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-0.97755061,  0.08922032,  0.90504565, -1.05338635,
         -0.69772374,  0.78468276, -0.60901197,  0.64628884,
          0.44063783, -1.06885905,  1.21006289, -1.06753995,
          0.85796332,  1.14981742, -0.61003589,  1.37555295,
         -0.63366661, -0.33633854,  1.46606085,  1.55205843,
         -1.89197197,  1.15569582,  1.0168078 ,  0.60487545,
          0.68514464,  0.94068731,  1.55821276,  1.80323696,
          1.35601575,  1.46810839,  1.66070328,  1.16286603,
          0.60284565, -1.89413413,  1.52102876,  1.17974371,
          1.36205135,  0.93842961, -1.7525565 , -1.78966564,
         -1.61127222, -0.55868553, -0.65956316,  0.46182883,
         -0.18821393,  2.10693108,  1.35866401,  1.78598251,
          1.89015757,  1.46935856, -1.43651044,  1.35689633,
          1.18202414,  0.19068774,  1.48293979,  1.48377853,
          1.53635924,  1.66809735,  1.39996042,  1.37876519,
          1.63347986,  2.00963311, -0.82061447,  2.39668535,
         -1.22330659, -1

In [73]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3268.310000
5     3466.227000
6     3246.749286
7     3017.084229
8     2776.217486
9     2891.489000
10    2976.485000
11    3258.027000
12    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [74]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

12    3537.032
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [75]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Santa Catarina - IDH Renda,Santa Catarina - IDH Longevidade,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Santa Catarina - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
13,-1.058114,-0.526718,0.605313,-0.874364,-1.628833,0.37534,-0.67958,1.051516,-0.181112,1.163167,...,-1.522321,-2.313421,0.942623,-1.951417,0.77328,0.392088,-0.143561,-2.013906,-1.686084,0.21068


In [76]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-1.05811448, -0.52671751,  0.60531339, -0.87436433,
         -1.62883284,  0.37533993, -0.67957965,  1.05151628,
         -0.18111229,  1.16316662,  2.6648078 , -0.28415403,
          0.78842482,  2.70456197, -0.13731248,  1.0620003 ,
          0.14874796, -0.17746697,  1.94394596,  2.11975022,
         -2.30353113,  1.50318835,  0.92591867,  0.21976702,
          0.38034066,  1.07012252,  2.38027465,  1.18721136,
          0.68454647,  0.96357222,  1.49963583,  1.10064062,
          1.54938258, -1.601903  ,  1.82071202,  1.75021424,
          1.09338904,  0.63049382, -1.89207081, -2.1907463 ,
         -2.64430562,  0.34649211, -1.64566109,  0.29105292,
         -0.8938401 ,  1.60067003,  1.08600407,  1.57190372,
          1.48403903,  1.96565138, -2.59491426,  2.94159734,
          1.66925951,  0.0371629 ,  1.62436238,  1.59912894,
          1.82219669,  1.96888178,  1.62404736,  1.59415473,
          1.92575804,  2.73371511,  1.06901998,  1.0388819 ,
         -1.5223207 , -2

In [77]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

13    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [78]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(592, activation='tanh', return_sequences=True, input_shape=(train_input.shape[1], 
                                                                                         train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=5000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [79]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, 0, seed)

        current_loss = min(history.history['loss'])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [80]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1825823317, 1457847336, 435966045, 1989067430, 1826781009, 3688723134, 3901228695, 1954390817, 194962020, 1015378005, 2309626476, 1240074928, 1569529584, 1701535389, 3526158183, 2864914429, 3935708935, 1433310210, 3358841353, 2063563807, 2214132680, 1405757991, 2345326445, 2893583790, 1642596628, 121494933, 2991153044, 778249175, 2455104444, 1890730436, 1787848934, 3264058927, 3734150469, 2470856321, 1518308048, 2380311642, 1288863436, 2870074175, 995064480, 3082772225, 4128634982, 62123970, 2867047166, 3890728484, 666111247, 2878119640, 1746045002, 2004755615, 1436670087, 4058030564, 1203494342, 1282194716, 1314754203, 1358391275, 284244837, 3095925463, 4223965429, 2595648756, 1277440452, 542168896, 2813711683, 3352243487, 3840835664, 2057298525, 3378023534, 859485620, 865227668, 4138908802, 2131588445, 1964889983, 1293975162, 2087942211, 3048939467, 4288438995, 1152376721, 1516517378, 889731549, 2378944435, 3737040424, 2690565056, 476880132, 1576664636, 1702121397, 755503345, 345379

2023-09-02 11:26:32.120107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:26:32.121755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:26:32.123086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 15276.69921875
winner_seed: 1825823317


Step: 1 ___________________________________________


2023-09-02 11:28:00.825350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:28:00.831981: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:28:00.833464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 13825.7353515625
winner_seed: 1457847336


Step: 2 ___________________________________________


...

loss: 16688.572265625


Step: 98 ___________________________________________


2023-09-02 12:39:36.615388: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:39:36.616691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:39:36.617716: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 35539.734375


Step: 99 ___________________________________________


2023-09-02 12:40:10.930747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:40:10.934065: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:40:10.938112: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 7743.0244140625


final_seed: 1826781009


In [81]:
trained_model, history = lstm_model(reshaped_train_input, 
                                    train_target, want_verbose=1, seed=winner_seed)

2023-09-02 12:41:37.786720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:41:37.787981: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:41:37.789251: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5000


2023-09-02 12:41:38.552263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:41:38.555174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:41:38.562755: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 4s 4s/step - loss: 9682634.0000
Epoch 2/5000
1/1 [==============================] - 0s 15ms/step - loss: 9430468.0000
Epoch 3/5000
1/1 [==============================] - 0s 14ms/step - loss: 1559776.7500
Epoch 4/5000
1/1 [==============================] - 0s 14ms/step - loss: 182482.0312
Epoch 5/5000
1/1 [==============================] - 0s 14ms/step - loss: 188287.0000
Epoch 6/5000
1/1 [==============================] - 0s 13ms/step - loss: 131248.9844
Epoch 7/5000
1/1 [==============================] - 0s 14ms/step - loss: 95663.8359
Epoch 8/5000
1/1 [==============================] - 0s 14ms/step - loss: 76116.8750
Epoch 9/5000
1/1 [==============================] - 0s 15ms/step - loss: 65509.2305
Epoch 10/5000
1/1 [==============================] - 0s 14ms/step - loss: 74552.0469
Epoch 11/5000
1/1 [==============================] - 0s 14ms/step - loss: 65619.7500
Epoch 12/5000
1/1 [==============================] - 0s 13ms/step - loss: 70046.

1/1 [==============================] - 0s 15ms/step - loss: 56932.7109
Epoch 98/5000
1/1 [==============================] - 0s 16ms/step - loss: 43937.0898
Epoch 99/5000
1/1 [==============================] - 0s 15ms/step - loss: 51958.4219
Epoch 100/5000
1/1 [==============================] - 0s 14ms/step - loss: 49262.5781
Epoch 101/5000
1/1 [==============================] - 0s 17ms/step - loss: 42474.4570
Epoch 102/5000
1/1 [==============================] - 0s 16ms/step - loss: 44809.8398
Epoch 103/5000
1/1 [==============================] - 0s 17ms/step - loss: 61372.7695
Epoch 104/5000
1/1 [==============================] - 0s 15ms/step - loss: 41876.4922
Epoch 105/5000
1/1 [==============================] - 0s 13ms/step - loss: 46777.8164
Epoch 106/5000
1/1 [==============================] - 0s 15ms/step - loss: 23733.8730
Epoch 107/5000
1/1 [==============================] - 0s 14ms/step - loss: 54872.5234
Epoch 108/5000
1/1 [==============================] - 0s 16ms/step - lo

...

1/1 [==============================] - 0s 17ms/step - loss: 41661.2656
Epoch 1329/5000
1/1 [==============================] - 0s 18ms/step - loss: 35583.7031
Epoch 1330/5000
1/1 [==============================] - 0s 19ms/step - loss: 51148.4414
Epoch 1331/5000
1/1 [==============================] - 0s 20ms/step - loss: 49563.2148
Epoch 1332/5000
1/1 [==============================] - 0s 21ms/step - loss: 45753.4414
Epoch 1333/5000
1/1 [==============================] - 0s 24ms/step - loss: 40021.6914
Epoch 1334/5000
1/1 [==============================] - 0s 21ms/step - loss: 56819.3555
Epoch 1335/5000
1/1 [==============================] - 0s 19ms/step - loss: 39843.0938
Epoch 1336/5000
1/1 [==============================] - 0s 24ms/step - loss: 40309.0391
Epoch 1337/5000
1/1 [==============================] - 0s 22ms/step - loss: 47517.7266
Epoch 1338/5000
1/1 [==============================] - 0s 20ms/step - loss: 36003.9609
Epoch 1339/5000
1/1 [==============================] - 0s 2

1/1 [==============================] - 0s 19ms/step - loss: 77324.9141
Epoch 1423/5000
1/1 [==============================] - 0s 18ms/step - loss: 49210.5859
Epoch 1424/5000
1/1 [==============================] - 0s 19ms/step - loss: 61304.4414
Epoch 1425/5000
1/1 [==============================] - 0s 24ms/step - loss: 67134.0234
Epoch 1426/5000
1/1 [==============================] - 0s 19ms/step - loss: 31467.9043
Epoch 1427/5000
1/1 [==============================] - 0s 20ms/step - loss: 60801.3555
Epoch 1428/5000
1/1 [==============================] - 0s 18ms/step - loss: 60271.7695
Epoch 1429/5000
1/1 [==============================] - 0s 19ms/step - loss: 46602.3203
Epoch 1430/5000
1/1 [==============================] - 0s 20ms/step - loss: 46948.7305
Epoch 1431/5000
1/1 [==============================] - 0s 21ms/step - loss: 30882.4805
Epoch 1432/5000
1/1 [==============================] - 0s 18ms/step - loss: 75677.5625
Epoch 1433/5000
1/1 [==============================] - 0s 2

1/1 [==============================] - 0s 17ms/step - loss: 42266.2539
Epoch 1517/5000
1/1 [==============================] - 0s 18ms/step - loss: 74798.0703
Epoch 1518/5000
1/1 [==============================] - 0s 20ms/step - loss: 50651.9023
Epoch 1519/5000
1/1 [==============================] - 0s 19ms/step - loss: 42725.0352
Epoch 1520/5000
1/1 [==============================] - 0s 22ms/step - loss: 59615.9805
Epoch 1521/5000
1/1 [==============================] - 0s 20ms/step - loss: 50599.8359
Epoch 1522/5000
1/1 [==============================] - 0s 18ms/step - loss: 50289.4102
Epoch 1523/5000
1/1 [==============================] - 0s 21ms/step - loss: 65535.0273
Epoch 1524/5000
1/1 [==============================] - 0s 24ms/step - loss: 57803.2500
Epoch 1525/5000
1/1 [==============================] - 0s 28ms/step - loss: 43090.2031
Epoch 1526/5000
1/1 [==============================] - 0s 19ms/step - loss: 65620.9609
Epoch 1527/5000
1/1 [==============================] - 0s 2

1/1 [==============================] - 0s 15ms/step - loss: 43442.4336
Epoch 1611/5000
1/1 [==============================] - 0s 14ms/step - loss: 40262.7148
Epoch 1612/5000
1/1 [==============================] - 0s 14ms/step - loss: 39534.8906
Epoch 1613/5000
1/1 [==============================] - 0s 15ms/step - loss: 50566.8398
Epoch 1614/5000
1/1 [==============================] - 0s 13ms/step - loss: 70911.2891
Epoch 1615/5000
1/1 [==============================] - 0s 16ms/step - loss: 76752.6094
Epoch 1616/5000
1/1 [==============================] - 0s 14ms/step - loss: 80354.3047
Epoch 1617/5000
1/1 [==============================] - 0s 13ms/step - loss: 71494.1875
Epoch 1618/5000
1/1 [==============================] - 0s 14ms/step - loss: 68595.2266
Epoch 1619/5000
1/1 [==============================] - 0s 14ms/step - loss: 65840.3125
Epoch 1620/5000
1/1 [==============================] - 0s 15ms/step - loss: 64886.3359
Epoch 1621/5000
1/1 [==============================] - 0s 1

1/1 [==============================] - 0s 15ms/step - loss: 63603.5391
Epoch 1705/5000
1/1 [==============================] - 0s 13ms/step - loss: 64493.5547
Epoch 1706/5000
1/1 [==============================] - 0s 15ms/step - loss: 64335.7227
Epoch 1707/5000
1/1 [==============================] - 0s 14ms/step - loss: 64783.3672
Epoch 1708/5000
1/1 [==============================] - 0s 14ms/step - loss: 64324.2891
Epoch 1709/5000
1/1 [==============================] - 0s 14ms/step - loss: 64321.1055
Epoch 1710/5000
1/1 [==============================] - 0s 14ms/step - loss: 64319.3281
Epoch 1711/5000
1/1 [==============================] - 0s 13ms/step - loss: 64318.0664
Epoch 1712/5000
1/1 [==============================] - 0s 14ms/step - loss: 64317.0898
Epoch 1713/5000
1/1 [==============================] - 0s 13ms/step - loss: 64316.1836
Epoch 1714/5000
1/1 [==============================] - 0s 15ms/step - loss: 48283.2695
Epoch 1715/5000
1/1 [==============================] - 0s 1

In [82]:
prediction = trained_model.predict(reshaped_test_input)[0][0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

2023-09-02 12:42:16.138131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:42:16.140619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:42:16.142293: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 748ms/step
[4089.333](test_target) - [3281.1482](prediction) = 808.1848066406251


In [83]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.19763242725418181

In [84]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = lstm_model(dfToInputRNN(train_input), 
                                       dfToInputRNN(train_target), want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0][0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [85]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Santa Catarina - IDH Renda,Santa Catarina - IDH Longevidade,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - IDH,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Santa Catarina - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,1.237454,-1.239228,-0.932986,-1.099975,-0.729604,-0.816011,-0.465381,-0.899279,-1.223534,1.148205,...,1.308949,1.412346,-1.220012,1.323311,1.276699,0.891356,0.813979,-0.083359,-1.191078,0.945270
1,-0.025828,0.029499,-0.453916,-0.219779,-0.684368,-0.592292,1.389222,-0.495597,-0.002418,0.140893,...,-0.190804,-0.643250,-0.009412,-0.229640,-1.165161,-1.396527,0.594549,1.264295,1.255842,0.438326
2,-1.211627,1.209729,1.386902,1.319754,1.413972,1.408303,-0.923841,1.394876,1.225952,-1.289098,...,-1.118144,-0.769096,1.229424,-1.093670,-0.111538,0.505170,-1.408528,-1.180936,-0.064763,-1.383595


train_target:


,Santa Catarina - Consumo de Cimento (t)
0,2669.400
1,2852.759
2,3087.810


test_input:


...

test_target:


,Santa Catarina - Consumo de Cimento (t)
13,4089.333


2023-09-02 12:49:55.743655: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 12:49:55.745157: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 12:49:55.746311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 934ms/step
Error: 808.1848066406251




[3100.07275390625,
 3205.44189453125,
 3258.402099609375,
 3317.339111328125,
 3235.727294921875,
 3054.097412109375,
 2945.444091796875,
 2876.929443359375,
 2915.506591796875,
 3045.926025390625,
 3281.148193359375]

In [86]:
display(targets)
display(predictions)

[3268.31,
 3268.31,
 3466.227,
 3246.749285714286,
 3017.084228571429,
 2776.217485714286,
 2891.489,
 2976.485,
 3258.027,
 3537.032,
 4089.333]

[3100.07275390625,
 3205.44189453125,
 3258.402099609375,
 3317.339111328125,
 3235.727294921875,
 3054.097412109375,
 2945.444091796875,
 2876.929443359375,
 2915.506591796875,
 3045.926025390625,
 3281.148193359375]

In [87]:
mae = mean_absolute_error(predictions, targets)
mae

254.66953710937494

In [88]:
porcentage = mae/np.mean(targets)
porcentage

0.0782607695868125